# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key[:8]=='sk-proj-':
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key")

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [5]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [16]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example, add other relevant links beside the one provided in the example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [17]:
link_system_prompt

'You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.\nYou should respond in JSON as in this example, add other relevant links beside the one provided in the example:\n{\n    "links": [\n        {"type": "about page", "url": "https://full.url/goes/here/about"},\n        {"type": "careers page": "url": "https://another.full.url/careers"}\n    ]\n}\n'

In [18]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt 

In [19]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [20]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} 
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [23]:
anthropic = Website("https://anthropic.com")
anthropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/opus',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://ww

In [21]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/'},
  {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [24]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [25]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
So

In [40]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [29]:
get_brochure_user_prompt("Anthropic","https://anthropic.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'learning resources page', 'url': 'https://www.anthropic.com/learn'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nEducation\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certificat

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'API page', 'url': 'https://www.anthropic.com/api'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}


# Anthropic Company Brochure

## About Us
Anthropic is an AI safety and research company headquartered in San Francisco, dedicated to building reliable, interpretable, and steerable AI systems. Our mission is driven by the belief that AI will fundamentally impact the world, and we are determined to ensure that this impact is a positive one. Through rigorous research and development, we aim to create AI technologies that benefit humanity while responsibly addressing the associated risks.

### Our Core Values
1. **Act for Global Good**: Prioritizing long-term positive outcomes for humanity through bold action.
2. **Hold Light and Shade**: Recognizing the dual potential of AI to pose risks and create benefits.
3. **Be Good to Our Users**: Emphasizing kindness and generosity in all interactions with users and stakeholders.
4. **Ignite a Race to the Top on Safety**: Leading AI safety initiatives to inspire industry-wide improvements.
5. **Do the Simple Thing That Works**: Focusing on effective, practical solutions through empirical approaches.
6. **Be Helpful, Honest, and Harmless**: Maintaining a culture of direct and kind communication.
7. **Put the Mission First**: Ensuring collective efforts are aligned with our mission to enhance AI safety.

## Our Products
We focus on developing AI systems, notably our flagship model **Claude**, which includes various iterations:
- **Claude Opus 4**: Our most intelligent AI model to date.
- **Claude Sonnet 4**: Enhancing coding capabilities and AI agent performance.
- **Claude Haiku 3.5**: A balanced approach for practical applications across industries.

### Solutions Offered
- **AI Agents**: Collaborate with Claude for customer support and education.
- **Coding**: Create AI-powered applications seamlessly.
- **Enterprise Solutions**: Tailored packages for corporate needs.

## Research & Innovation
At Anthropic, research is a cornerstone of our operations. We conduct frontier research across various AI modalities, focusing on interpretability, reinforcement learning from human feedback, and the societal impacts of AI. Our findings feed directly into product development, creating a continuous feedback loop of learning and improvement.

## Our Team
Anthropic's diverse team comprises researchers, engineers, policy experts, and operational leaders united by a commitment to AI safety. Our collaborative environment allows us to leverage our varied backgrounds—ranging from physics and machine learning to public policy and business—to forecast the impact of our innovations.

## Company Culture
We pride ourselves on maintaining a high-trust, low-ego workplace culture where innovation and critical thinking flourish. Our team is passionate about ensuring AI technologies are developed responsibly, with an eye toward their potential impacts on society.

### Benefits & Career Opportunities
Anthropic recognizes that well-supported employees contribute to our success. Our benefits include:
- Comprehensive health, dental, and vision insurance
- Flexible paid time off and generous parental leave
- Competitive salary and equity packages
- Daily meals, wellness stipends, and education support

We are always looking for talented individuals who are passionate about creating safe AI. Interested candidates can explore our current openings on our [Careers Page](#).

## Join Us
Are you ready to help shape the future of safe AI? At Anthropic, you will work at the forefront of technology with like-minded individuals committed to maximizing the well-being of humanity through responsible AI development.

[Explore Our Open Roles](#)

---

*For more information about our products, initiatives, and career opportunities, visit our [website](#). Become a part of the movement towards safer AI today!*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [38]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    # for chunk in stream:
    #     print(chunk.choices[0].delta.content or '', end='')
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}



# Welcome to Anthropic: Where AI Gets a Safety Net!

**Are you ready to chat with Claude?** 💬  
**Meet our star AI assistant, Claude, who can do everything from coding to providing stellar customer support!** (No pressure, Claude!)

---

## **Why Anthropic? Because Humans Matter!**

At Anthropic, we believe AI should serve humanity, but hey, we like to have some fun doing it! Our mission is simple: build AI that’s not just smart but safe, steerable, and interpretable. Kind of like a nice, well-trained dog that doesn’t bite your mailman. 

### **What We Do:**

- **Build Frontier AI Systems:** We’re pushing the boundaries with models like Claude Opus 4, Sonnet 4, and Haiku 3.5. If you thought “twinkle twinkle little star” was a masterpiece, wait ‘til you see what we can do!
- **Collaborate for Good:** We’re all about teamwork! Our diverse group includes researchers, engineers, policy wizards, and even a resident plant enthusiast or two. 🌱

---

## **Join Us! Let’s Build a Safer Future.** 

We’re a **Public Benefit Corporation** with offices in *San Francisco*, where the sun shines, the AI shines brighter, and snacks are always nearby! 🍪

### **Careers at Anthropic: What’s in the Gig Bag?**

- **Health & Wellness:** Comprehensive medical, dental, & vision plans (because who wants to lose an eye while coding?).
- **Generous Parental Leave:** 22 weeks of paid leave – because parenting is also a full-time gig!
- **Flexible Time Off:** Take time when you need it, whether it’s for a family reunion, a tropical getaway, or just to binge-watch your favorite series. 🍿
- **Daily Meals & Snacks:** Fuel your brilliance with our kitchenette filled with snacks! We’ve got your back.

---

### **Values We Live By:**

1. **Act for the Global Good:** No evil robots on our watch! Only helpful, safe AI.
2. **Hold Light and Shade:** Balance is key! We’re serious about preventing disasters while creating fabulous tech.
3. **Be Good to Our Users:** From policy-makers to pets, we’re all about kindness! 🐾
4. **Ignite a Race to the Top on Safety:** We believe in pushing for the best, not just passing the lowest bar. Who wants a mediocre robot?
5. **Do the Simple Thing That Works:** Why reinvent the wheel when a bicycle will do? (Maybe with a cool robot horn.)
6. **Be Helpful, Honest, and Harmless:** We’re here to make waves, not disrupt the ocean (or your workflow).
7. **Put the Mission First:** AI for humanity, not for world domination! 

---

## **Want to Team Up?**
Ready to make history (and possibly a few great AI friends)? Check out our open roles [here](#) and apply! We guarantee a high-five from Claude and free snacks on your first day.

### **Contact us today**, and let’s make the AI world a safer place, one chat at a time! 🚀



In [42]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}]}



# 🤗 Welcome to Hugging Face! 🤗

## The AI community building the future 🌟

---

### Who Are We? 
At Hugging Face, we're just a bunch of machine learning (ML) enthusiasts who dream about AI in our sleep. No, seriously, our engineers often talk to their models like they're pets—only more predictable and a lot less likely to knock over a vase!

## What We Offer:
- **1M+ Models**: More models than your favorite pizza place has toppings! From text to images, audio to video, we've got it all. Want to translate a cat meme? We've got a model for that!
- **250k+ Datasets**: Data, glorious data! Your data dreams are about to come true—made fresh daily, no GMOs, all organic!
- **AI Spaces**: Create unique apps faster than you can say “AI revolution.” Our Spaces are like a sandbox, but far less likely to have a sandbox incident.
  
## Our Customers:
Join the crowd of over **50,000 organizations**! Yes, Google, Microsoft, and Amazon flirt with us too, but don’t worry, they still have their healthy distance. 🤓

## Hugging Face Enterprise:
- Offering **advanced platform solutions** for organizations with needs that are, let’s be honest, a bit more complex than deciphering the recipe for instant noodles.
- **Team Pricing** starting at just **$20/user/month** — perfect for those who like to share costs (and cookies!).

## Company Culture:
### Embrace Your Awkwardness 🎉
Join us where quirky greetings take place in GitHub comments and lunchtime discussions might just turn into debates about the greatest fictional AI sidekick. Spoiler: It's still Data from Star Trek.

### Hugging Face Careers:
Are you looking for a job so chill that you could nap during meetings? 
- **Open Positions**: Join the ranks of our ever-growing team! We promise not to let you hug the coffee machine all day. (Okay, maybe just a little.)

---

### Pricing Overview
*Because who doesn’t like numbers?*
- **PRO Plan** for individuals at $9/month.
- **Team Plan** for friends who work together at **$20/user/month**.
- **Enterprise Plans**: Contact sales for tailored solutions. No, we can’t help you convince your cat to love you in return, but we can help with ML!

---

### Join Us 
Ready to revolutionize the AI world (and maybe snag a few hugs while you’re at it)? **[Sign Up Now](#)** and let’s unleash the power of AI together!

**P.S.** If you’re not into hugs, we also accept high-fives!

---
You’ve reached the end of the brochure. Now go forth and be awesome!



In [43]:
stream_brochure("Infobip", "https://infobip.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.infobip.com/company'}, {'type': 'careers page', 'url': 'https://www.infobip.com/careers'}, {'type': 'contact page', 'url': 'https://www.infobip.com/contact'}, {'type': 'blog page', 'url': 'https://www.infobip.com/blog'}, {'type': 'downloads page', 'url': 'https://www.infobip.com/downloads'}, {'type': 'academy page', 'url': 'https://academy.infobip.com/'}, {'type': 'values page', 'url': 'https://www.infobip.com/company/values'}, {'type': 'leadership page', 'url': 'https://www.infobip.com/company/leadership'}, {'type': 'offices page', 'url': 'https://www.infobip.com/company/offices'}, {'type': 'media kit', 'url': 'https://www.infobip.com/media-kit'}, {'type': 'resources page', 'url': 'https://www.infobip.com/resources'}, {'type': 'news page', 'url': 'https://www.infobip.com/news'}]}



# Welcome to Infobip: Your Omnichannel Fairy Godmother!

**Where Every Customer Experience is a Happily Ever After!**

---

### 🥳 Who Are We?

Infobip is the superhero of omnichannel marketing, armed with a flashy array of powerful APIs ready to save your customer engagement woes! Whether it’s SMS, email, voice, or even carrier pigeons (just kidding!), we’ve got you covered in every realm of communication.

### 🚀 Our Mission

We help you reach every customer **wherever** they are (including that corner coffee shop with Wi-Fi!). Our goal? To engage your audience like never before while you sip your latte, or maybe green tea? (We don't discriminate on beverages!)

---

### 🛠️ What Do We Offer?

- **Conversational Experience Cloud:** Capture hearts (and wallets) with chatbot building platforms that make your customers feel like royalty!
- **Cloud Contact Center:** A fancy name for making sure your customers can talk to a real human (or super-intelligent bot) whenever they feel like it!
- **Customer Data Platform:** Like a Swiss Army knife for customer insights - and we guarantee no one will accidentally poke themselves!

---

### 🌈 Our Customers

From the finance nerds to retail rebels, we cater to every industry. Our clients include major players in **Healthcare**, **Marketing**, **Telecoms**, **Transportation**, and even **Government Solutions** (yes, we help them too!).

---

### 🎉 Company Culture

At Infobip, we believe in:

- **Work Hard, Play Hard:** (But don’t worry, we don’t have a mandatory beanbag chair or “nap-room” policy… yet).
- **Innovation**: We’re always on the lookout for the next cool thing – think of us as the Apple of messaging!
- **Team Spirit**: Yes, we believe in kindness and empathy to deliver those delightful customer experiences. No dragons here! 🐉

---

### 💼 Join the Infobip Family!

Interested in turning your career into a fairytale? We’re always on the lookout for creative minds, technical geniuses, and anyone who believes they can sprinkle some marketing magic! 

*Perks include:*

- **Flexible Hours**: Because who doesn’t want to work in pajamas sometimes?
- **Development Programs**: Because “growth” should apply to your career too!
- **Fun Events**: Who is up for a karaoke night? (You sing, we provide the feedback. Or earplugs.)

---

### 🎁 Try Us for Free!

Want to see if we fit like a glove? Give our omnichannel platform a whirl with our free trial! You won’t regret it—unless you don’t like fun. Then, maybe you should stick to life in a box.

---

### 📞 Get In Touch

Ready to dive into the world of omnichannel marketing that makes sense? Contact us, and let’s create magic together! ✨

---

**Infobip: The Wizards of Communication. Transforming Relationships, One Message at a Time!**



In [44]:
stream_brochure("Moxie", "https://www.joinmoxie.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.joinmoxie.com/about'}, {'type': 'careers page', 'url': 'https://www.joinmoxie.com/careers'}, {'type': 'blog', 'url': 'https://www.joinmoxie.com/blog'}, {'type': 'press reviews', 'url': 'https://www.joinmoxie.com/press-reviews'}, {'type': 'training', 'url': 'https://www.joinmoxie.com/training'}, {'type': 'how we work', 'url': 'https://www.joinmoxie.com/how-we-work'}, {'type': 'clinical advisory board', 'url': 'https://www.joinmoxie.com/clinical-advisory-board'}]}



# Welcome to Moxie: Where Aesthetic Dreams Come True!

### 🥳 A MedSpa in a Month? Yes, Please!

Are you ready to turn your medspa dreams into a reality? At Moxie, we offer an **All-in-One MedSpa Software**, because "multi-tasking" is so last season! With our magical blend of **Marketing, Compliance,** and **Business Coaching**, you’ll be up, running, and hopefully rolling in new clients before your morning coffee gets cold! ☕✨

---

### 💼 Who Trusts Moxie? (Is It You?)

Join the ranks of 400+ savvy medspa owners who took the leap and launched their practices with Moxie’s *expert guidance*. 

* "Before Moxie, I thought opening a medspa was impossible. Now, look at me—I've got more clients than I can handle, and I owe it all to this fantastic team!" - **The Thrilled Entrepreneur**

* (Warning: Actual quotes may contain remnants of excitement and spontaneous ice-cream eating!)

---

### 🎉 Moxie Culture: All about YOU!

At Moxie, you matter more than the last slice of pizza at a staff meeting! (Well, almost.) Our team is dedicated to your success. Think of us as your medspa *Gurus* who’ll hold your hand through thick and thin (and all layers of dermal filler).  With experts like Nicole, our legal eagle, we know our compliance stuff inside and out, so you can keep your license safe and sound.

---

### 🚀 Medspa Launch & Growth (Without the Stress!)

- **Launch:** We take your medspa from “what if” to “wow”—in just 30 days! 
- **Run:** Say goodbye to chaotic paperwork and hello to profitability with our intuitive software.
- **Grow:** We help you maximize profits, minimize waste, and create the kind of raving fan experiences that keep patients coming back for more!

---

### 🌟 Careers at Moxie: Join the Fun!

Looking for a workplace that’s as revitalizing as a fresh moisturizer? Moxie is on the lookout for enthusiastic go-getters ready to help medspa owners thrive! 

Whether you’re a strategy wizard, a compliance ninja, or just someone who loves helping others succeed, we want YOU on our team! (No experience with needles necessary!)

---

### 💡 Get Started Today—No Strings Attached!

Remember, it’s all about your *medspa vision*! A free strategy session with our advisors could be your first step toward owning the medspa of your dreams. We promise no hard sells—just clarity and cake (well, figuratively speaking)!

👉 [**Book Your Free Strategy Session Now!**](#)

*Disclaimer: Our cake may or may not be calorie-free, but your medspa growth sure will be rewarding!*

---

Let’s turn your medspa dreams into reality with Moxie—because why *just* survive when you can thrive?



## Business Applications

In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.

In [58]:
def create_brochure_pretranslate(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

def create_brochure_in_Spanish(company_name, url):
    brochure = create_brochure_pretranslate(company_name, url)
    prompt = f"Translate the provided information below to Spanish:\n\n{brochure}"
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are the best translator in the world."},
            {"role": "user", "content": prompt}
        ],
    )
    
    result = response.choices[0].message.content
    display(Markdown(result))


In [59]:
create_brochure_in_Spanish("infobip", "https://infobip.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.infobip.com/company'}, {'type': 'careers page', 'url': 'https://www.infobip.com/careers'}, {'type': 'contact page', 'url': 'https://www.infobip.com/contact'}, {'type': 'blog', 'url': 'https://www.infobip.com/blog'}, {'type': 'values page', 'url': 'https://www.infobip.com/values'}, {'type': 'offices page', 'url': 'https://www.infobip.com/offices'}, {'type': 'partners page', 'url': 'https://partners.infobip.com'}, {'type': 'resources page', 'url': 'https://www.infobip.com/resources'}, {'type': 'events page', 'url': 'https://www.infobip.com/events'}, {'type': 'media kit', 'url': 'https://www.infobip.com/media-kit'}]}


# 📬 ¡Bienvenido a Infobip! 🌍

### ¡La Mejor Plataforma de Marketing Omnicanal en la Ciudad!

**¿Por qué Infobip?** 🤔  
Nos aseguramos de que tus clientes estén a solo un mensaje de distancia. No importa dónde estén o qué canal prefieran, Infobip entrega mensajes más ruidosos que una banda de metales en un retiro silencioso.

---

### **Nuestra Cultura:**

En Infobip, creemos en **"Marketing que se Escucha Claro y Fuerte!"**  
Valoramos la creatividad, la innovación y el tipo de espíritu de equipo que te podría conseguir una medalla de oro en una carrera de relevos. ¡Nuestra oficina no es solo un espacio de trabajo; es una comunidad vibrante donde las ideas fluyen tan libremente como el café en las reuniones de los lunes por la mañana!

- **Trabajo en Equipo**: Todo el mundo dice que no hay “yo” en “equipo”, ¡pero preferimos pensarlo como “Ganar Juntos”!
- **Innovación**: Nuestras ideas son tan frescas como tu tostada de aguacate por la mañana – y igual de populares (probablemente).

---

### **Nuestros Clientes:**

Atendemos a héroes como:
- **Telecomunicaciones**: Elevando la comunicación para los superhéroes de la conectividad.
- **Finanzas**: Manteniendo tus transacciones tan seguras como una caja fuerte custodiada por un dragón.
- **Salud**: Porque incluso tu doctor quiere enviarte un mensaje más rápido que una bala tras una cita.
- **Retail y eCommerce**: ¡Transformando navegadores en compradores más rápido de lo que puedes decir “¡Venta Flash!”!

---

### **Únete a Nosotros: ¡La Familia Infobip!** 🤝

¿Estás listo para aportar tus superpoderes únicos a nuestro equipo? Aquí hay solo algunas razones por las que deberías:

- **Entorno Laboral Innovador**: Aceptamos ideas frescas más rápido de lo que podemos decir “API.” (¡Y lo decimos mucho!)
- **Desarrollo Profesional**: Nuestro crecimiento es exponencial; ¡únete a nosotros y podrías encontrarte subiendo de nivel como un personaje de videojuego!
- **Diversión Después del Trabajo**: ¿Alguna vez has intentado crear una estrategia de marketing digital mientras juegas al ping-pong? ¡Ahora es tu oportunidad!

---

### **Lo Que Ofrecemos:**
- 🌟 **Soluciones Conversacionales**: Porque a nadie le gusta hablar con robots (a menos que sean realmente, realmente inteligentes).
- 🚀 **Canales Programables**: Desde SMS hasta Instagram, ¡te cubrimos!
- 📞 **Centro de Contacto en la Nube**: Es como si tu equipo de soporte al cliente volara a la luna... ¡sin el cohete!
- 🤖 **Chatbots de IA**: Nuestros chatbots conversan mejor que tu amigo más sociable.

---

### **¡Ponte en Contacto! 🚀**

¿Quieres enviar un mensaje que importe? ¡Contáctanos hoy!
- 📩 [Pruébalo GRATIS](#)
- 📞 ¡Llama a tu superhéroe más cercano... um, queremos decir línea de atención al cliente!

---

**Infobip – Conectando el Mundo, Un Mensaje a la Vez! 🌐💬**

*¿Te sientes intrigado? ¡Quizás seas nuestro próximo superhéroe! ¡Aplica ahora!*

*Descargo de responsabilidad: No se dañó a ningún superhéroe real durante la creación de nuestros servicios. 🚫🦸‍♂️*